## 기본적인 RAG 방법론 소개
- Embeddings에 대한 설명
- OpenAI Embeddings API

### 단어 기반 Retrieval 방법론의 한계
- 단어 기반 방법론은 단어의 의미나 문맥을 고려하지 않음
  - 따라서 동의어나 다의어를 구분 할 수 없음
- 단어를 의미를 가진 숫자로 표현하는 방법론 = 임베딩
  - 아래는 예시 숫자들이며 절대적인 숫자들을 매번 바뀔 수 있으나 비슷할 수록 비슷한 값들을 가지게 됨
    - "사과" = [0.2, -0.5, 0.7]
    - "배" = [0.18, -0.48, 0.65]
    - "컴퓨터" = [-0.6, 0.3, -0.2]
  - 위 숫자들은 일련의 딥러닝 모델의 학습 과정을 통해 거의 랜덤한 숫자들에서 시작해서 비슷한 단어들끼리 비슷한 값들을 가지게 됨
- 현재는 Transformer 구조 기반으로 학습된 딥러닝 모델 기반 임베딩의 성능이 가장 높은 편
  - 최근에는 아예 LLM 구조를 그대로 활용해서 임베딩을 생성
  - 성능은 가장 높을 수 있으나 비용이나 속도 면에서 비효율적일 수 있음

### Embeddings을 사용할 떄 알아야 할 점들
- (1) 차원 수가 높을 수록 (보통은) 더 성능이 높음
  - "사과"를 [0.2, -0.5, 0.7]로 표현
    - 3개의 숫자로 표현
    - 3차원 임베딩
  - 128 차원인 경우 128개의 숫자로 표현
  - 보통 256 ~ 4096 사이의 차원을 사용하고 384 ~ 1024 사이가 가장 흔히 사용되는 편
- (2) 차원 수가 높을 수록 연산 시간이 더 오래 걸리고 API 비용도 더 높음

### RAG의 가장 흔히 사용되는 Embeddings
- OpenAI나 Google은 ChatGPT이 Cutoff 이후의 데이터는 모른다는 점을 보완하기 위해 Embeddings API를 별도로 지원
  - Anthropic은 별도의 Embeddings API는 없어 타사 API 추천 (ex. Voyager)

### OpenAI Embeddings API
- 메인으로 2개의 임베딩 모델 제공 text-embedding-3-small, text-embedding-3-large
  - 임베딩 모델에 한해서는 GPT-4 만큼 압도적으로 좋진 않으나 범용적으로는 사용 가능한 수준
- text-embedding-3-small은 1536 차원, text-embedding-3-large는 3072 차원

In [1]:
# https://platform.openai.com/docs/models/embeddings
import os

from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

embedding = client.embeddings.create(
    input='내일 날씨는 어때?',
    model='text-embedding-3-small'
)

embedding

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.020379923284053802, 0.005209940019994974, -0.009987633675336838, -0.0017404785612598062, 0.03643738850951195, -0.005936480592936277, 0.03761456906795502, 0.014383664354681969, -0.03371516242623329, 0.00035666016628965735, -0.012167255394160748, 0.03906765207648277, -0.06456094980239868, -0.045468565076589584, -0.0006909033400006592, -0.014475631527602673, -0.06537026166915894, 0.029944509267807007, 0.01594710536301136, 0.014567598700523376, -0.014972254633903503, -0.030864180997014046, -0.01603907346725464, 0.0302388034760952, -0.029834147542715073, 0.013096123933792114, -0.038626208901405334, 0.022844644263386726, 0.002873973920941353, -0.016636859625577927, -0.0120936818420887, -0.031084900721907616, 0.01769448257982731, -0.021428348496556282, 0.03172867000102997, 0.011578666046261787, 0.02827070653438568, 0.02957664057612419, -0.04068627208471298, -0.014631975442171097, -0.018503792583942413, -0.008139094337821007, -0.01158786285

In [2]:
embedding.data[0].embedding

[-0.020379923284053802,
 0.005209940019994974,
 -0.009987633675336838,
 -0.0017404785612598062,
 0.03643738850951195,
 -0.005936480592936277,
 0.03761456906795502,
 0.014383664354681969,
 -0.03371516242623329,
 0.00035666016628965735,
 -0.012167255394160748,
 0.03906765207648277,
 -0.06456094980239868,
 -0.045468565076589584,
 -0.0006909033400006592,
 -0.014475631527602673,
 -0.06537026166915894,
 0.029944509267807007,
 0.01594710536301136,
 0.014567598700523376,
 -0.014972254633903503,
 -0.030864180997014046,
 -0.01603907346725464,
 0.0302388034760952,
 -0.029834147542715073,
 0.013096123933792114,
 -0.038626208901405334,
 0.022844644263386726,
 0.002873973920941353,
 -0.016636859625577927,
 -0.0120936818420887,
 -0.031084900721907616,
 0.01769448257982731,
 -0.021428348496556282,
 0.03172867000102997,
 0.011578666046261787,
 0.02827070653438568,
 0.02957664057612419,
 -0.04068627208471298,
 -0.014631975442171097,
 -0.018503792583942413,
 -0.008139094337821007,
 -0.011587862856686115,

In [3]:
len(embedding.data[0].embedding)

1536

In [4]:
embedding.usage

Usage(prompt_tokens=13, total_tokens=13)

### Embeddings API를 활용해서 텍스트 간의 유사도 구하기
- 두 텍스트 간의 얼마나 "비슷"한 지는 Cosine Similarity 또는 코사인 유사도 사용
- Cosine Similarity는 -1 ~ 1 사이로 표현되며 1은 완전히 같고, 0은 관련 없으며, -1은 정반대를 가리킴
  - Cosine Simialrity 계산을 위해서는 numpy라는 연산을 아주 빠르게 해주는 라이브러리 사용

In [5]:
import numpy as np


def get_embedding(text, model='text-embedding-3-small'):
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


text1 = "오늘은 날씨가 좋아서 공원에 산책을 갔습니다."
text2 = "날씨가 맑아서 야외 활동하기 좋은 날이었습니다."

embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)

similarity = cosine_similarity(embedding1, embedding2)

print(f"두 텍스트의 유사도: {similarity}")

두 텍스트의 유사도: 0.4159765531425354


In [6]:
embedding1 = get_embedding(text1, model='text-embedding-3-large')
embedding2 = get_embedding(text2, model='text-embedding-3-large')

similarity = cosine_similarity(embedding1, embedding2)

print(f"두 텍스트의 유사도: {similarity}")

두 텍스트의 유사도: 0.6626894164947986


### OpenAI Embeddings API를 활용해서 관련 있는 텍스트를 Retrieve하는 예시

In [7]:
text1 = """(종합) 오늘 낮까지 비 곳, 오늘 강풍과 풍랑 유의, 내일 늦은 오후~저녁 경기동부 소나기, 모레 낮부터 비
○ (오늘, 30일) 대체로 흐림, 서울.경기도(경기남서부 제외) 중심 낮(12~15시)까지 비 곳
○ (내일, 7월 1일) 구름많음, 대기불안정으로 늦은 오후(15~18시)부터 저녁(18~21시) 사이 경기동부 소나기 곳, 서해5도 대체로 맑다가 저녁부터 구름많아짐
○ (모레, 7월 2일) 대체로 흐림, 서울.인천.경기도 낮(12~15시)부터 비, 서해5도(연평도 부근) 늦은 밤(21~24시) 가끔 비 곳
* 예상 강수량(30일 낮까지)
- 서울.경기도(경기남서부 제외): 5mm 미만
* 소나기에 의한 예상 강수량(7월 1일 늦은 오후~저녁)
- 경기동부: 5~20mm
* 예상 강수량(7월 2일 낮~)
- 서울.인천.경기도: 10~50mm
- 서해5도(7월 2일 밤): 5~20mm"""

text2 = """서울의 주요 관광지 안내:
서울은 한국의 수도로서 다양한 문화와 역사를 자랑하는 도시입니다. 다음은 서울에서 꼭 가봐야 할 주요 관광지입니다.

경복궁: 조선 시대의 대표적인 궁궐로, 한국 전통 건축의 아름다움을 느낄 수 있습니다.
남산타워: 서울의 전경을 한눈에 볼 수 있는 전망대입니다. 케이블카를 타고 올라가면 더욱 즐거운 경험이 됩니다.
인사동: 전통 공예품과 예술 작품을 구경할 수 있는 거리로, 다양한 전통 음식점과 카페도 즐길 수 있습니다.
홍대: 젊음의 거리로, 다양한 카페, 레스토랑, 그리고 스트리트 공연을 즐길 수 있습니다.
강남: 현대적이고 세련된 분위기의 지역으로, 쇼핑과 미식 탐험에 최적의 장소입니다.
서울을 여행하기 좋은 시기는 봄과 가을입니다. 날씨가 쾌적하고, 다양한 축제와 이벤트가 열리기 때문입니다. 여행을 계획할 때는 서울의 대중교통 시스템을 잘 활용하면 편리하게 이동할 수 있습니다.
"""

question = '내일 날씨는 어때?'

In [8]:
embedding_question = get_embedding(question)
embedding_text1 = get_embedding(text1)
embedding_text2 = get_embedding(text2)

similarity1 = cosine_similarity(embedding_question, embedding_text1)
similarity2 = cosine_similarity(embedding_question, embedding_text2)

similarity1, similarity2

(0.35876081469945786, 0.1861478178948842)

- 첫 번째 텍스트가 질문과 가장 유사함을 알 수 있음
  - 절대적인 숫자보다는 주어진 텍스트 중에서 가장 높은 지가 중요함

In [9]:
embedding_question = get_embedding(question, model='text-embedding-3-large')
embedding_text1 = get_embedding(text1, model='text-embedding-3-large')
embedding_text2 = get_embedding(text2, model='text-embedding-3-large')

similarity1 = cosine_similarity(embedding_question, embedding_text1)
similarity2 = cosine_similarity(embedding_question, embedding_text2)

similarity1, similarity2

(0.368090433203833, 0.23126885738292308)